In [ ]:
import pandas as pd
from datetime import datetime
import pymongo
from pymongo import MongoClient
from bson.son import SON

In [ ]:
client = MongoClient('localhost', 27017)
db = client.DataManagement
treni_regionali = db.TreniRegionali
altri_treni = db.AltriTreni

Creazione del campo _numero_fermate_.

In [ ]:
pipeline = [{"$project":
                {"numero_fermate": { "$cond": { "if": {"$isArray": "$fermate"},
                                                "then": {"$size": "$fermate"}, "else": None}}
                }}]

lista_regionali = list(treni_regionali.aggregate(pipeline))
lista_altri_treni = list(altri_treni.aggregate(pipeline))

for elemento in lista_regionali :
    indice = elemento["_id"]
    numero_fermate = elemento["numero_fermate"]
    treni_regionali.find_one_and_update({"_id": indice}, {"$set": {"numero_fermate": numero_fermate}})

for elemento in lista_altri_treni:
    indice = elemento["_id"]
    numero_fermate = elemento["numero_fermate"]
    altri_treni.find_one_and_update({"_id": indice}, {"$set": {"numero_fermate": numero_fermate}})

Creazione del campo _ritardo_medio_treno_.

In [ ]:
pipeline = [{"$unwind":"$fermate"},
            {"$group": {"_id": {"id": "$_id", "numero_treno": "$numero_treno",
                                "id_origine": "$id_origine", "id_destinazione": "$id_destinazione"},
                                "ritardo_medio_treno": {"$avg": {"$cond":
                                                            [{"$eq":["$fermate.tipo_fermata", "P"]},
                                                            "$fermate.ritardo_partenza",
                                                            "$fermate.ritardo_arrivo"]
                                                            }}}}]

lista_regionali = list(treni_regionali.aggregate(pipeline))
lista_altri_treni = list(altri_treni.aggregate(pipeline))

for elemento in lista_regionali :
    indice = elemento["_id"]["id"]
    ritardo_medio_treno = elemento["ritardo_medio_treno"]
    treni_regionali.find_one_and_update({"_id": indice}, {"$set": {"ritardo_medio_treno": ritardo_medio_treno}})

for elemento in lista_altri_treni:
    indice = elemento["_id"]["id"]
    ritardo_medio_treno = elemento["ritardo_medio_treno"]
    altri_treni.find_one_and_update({"_id": indice}, {"$set": {"ritardo_medio_treno": ritardo_medio_treno}})

Calcolo del ritardo medio per ogni treno regionale, ordinato in ordine decrescente.

In [ ]:
pipeline = [{"$group": {"_id": {"numero_treno":"$numero_treno",
                        "id_origine":"$idOrigine",
                        "id_destinazione":"$idDestinazione"},
                "ritardo_medio": {"$avg": "$ritardo_finale"}}},
            {"$sort": SON ([("ritardo_medio", -1)])}]

lista_ritardi_regionali = pd.DataFrame(list(treni_regionali.aggregate(pipeline)))

Calcolo del ritardo medio per ogni stazione.

In [ ]:
pipeline = [{"$unwind":"$fermate"},
            {"$group": {"_id": "$fermate.nome_stazione",
                        "binari": {"$first": "$fermate.binari_tot_stazione"},
                        "ritardo_medio_stazione": {"$avg": {"$cond":
                                                            [{"$eq":["$fermate.tipo_fermata", "P"]},
                                                            "$fermate.ritardo_partenza",
                                                            "$fermate.ritardo_arrivo"]
                                                            }}}}]

lista_ritardi_regionali_stazione = pd.DataFrame(list(treni_regionali.aggregate(pipeline)))
lista__ritardi_altri_treni_stazione = pd.DataFrame(list(altri_treni.aggregate(pipeline)))